## **MoMA Collection Data Processing**
## Table of Contents
### 1. [General Cleaning](#general)
### 2. [Cleaning Punctuation](#punctuation)
### 2. [Cleaning Dates](#dates)
### 3. [Extracting from Biographies](#bio-extraction)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('./Artworks.csv')

## General Cleaning <a class='anchor' id='general'></a>

### Cleaning Punctuation <a class="anchor" id="punctuation"></a>

In [3]:
df['Nationality'].fillna('Nationality unknown',inplace=True)
df.fillna('-', inplace=True)
df['Cataloged'].replace({'Y':True,'N':False}, inplace=True)

df['cleanedDate'] = 0
df['Date2'] = 0

In [4]:
def strip_punct(df: pd.DataFrame, cols: [str, list], punct: str) -> pd.DataFrame:
    ''' 
    Takes in dataframe, column name or list of columns, and regex string of characters to remove
    overwrites original column with corrected column
    '''
    if type(cols) == str:
        df.loc[:,cols] = df.loc[:,cols].str.replace(f'{punct}','',regex=True)
    if type(cols) == list:
        for i in cols:
            df.loc[:,i] = df.loc[:,i].str.replace(f'{punct}','',regex=True)

In [5]:
strip_punct(df,['BeginDate','EndDate','Date','Gender'],'[^0-9\-]')
strip_punct(df,'Nationality','[^A-Za-z0-9\,\-\s]')
strip_punct(df,'ArtistBio','[\(\)]')

### Cleaning Dates <a class="anchor" id="dates"></a>

In [6]:
# adding properly formatted dates to new column
clean_dates = df[(df.Date.str.len()==4)&(df.Date.str.contains('\D')==False)].index
df.loc[clean_dates,'cleanedDate'] = df.loc[clean_dates,'Date']

print('{:,} total rows \n{:,} rows have dates correctly formatted'.format(len(df), len(df[df.Date.str.len()==4])))
print(f'completeness of Date {len(clean_dates)/(len(df)):.0%}\n')

140,848 total rows 
105,575 rows have dates correctly formatted
completeness of Date 75%



In [7]:
# identifying most common, alternate Date formatting
df[df.Date.str.len()!=4]['Date'].value_counts()[0:2]; #missing values
df[df.Date.str.len()!=4]['Date'].value_counts()[2:30];

In [8]:
df[df.Date.str.len()!=4]['Date'].value_counts()[0:2].sum(), df[df.Date.str.len()!=4]['Date'].value_counts()[0:2].sum()/len(df) #missing values

(3148, 0.02235033511302965)

- Identified 3,148 or 2% of records that are missing `Date` value - see [date imputation](#date-imputation)
- We can also capture a large share with formattings:
  - *YYYYYYYY*
  - *YYYY-YYYY*
  - *YYYY-YY*
  - later noticed and added *YYYY* which introduced an additional 2% improvement

In [9]:
# cleaning date ranges yyyyyyyy and yyyyyy
eight_digits = df[((df.Date.str.len()==6)|(df.Date.str.len()==8))&(df.Date.str.contains('\D',regex=True)==False)].index

df.loc[eight_digits,'Date2'] = df.loc[eight_digits,'Date'].apply(lambda x: str(x)[4:])
df.loc[eight_digits,'cleanedDate'] = df.loc[eight_digits,'Date'].apply(lambda x: str(x)[0:4])
print(f'YYYYYYYY & YYYYYY formatting: completeness of Date improved {len(eight_digits)/(len(df)):.0%}')

# # pulling all dates with ranges yyyy-yyyy | yyyy-yy
dash_ranges = df[((df.Date.str.len()==7)|(df.Date.str.len()==9))&(df.Date.str.contains('\d{4}\-\d{1,4}',regex=True))].index
df.loc[dash_ranges,'Date2'] = df.loc[dash_ranges,'Date'].apply(lambda x: x.split('-')[1])
df.loc[dash_ranges,'cleanedDate'] = df.loc[dash_ranges,'Date'].apply(lambda x: x.split('-')[0])
print(f'YYYY-YYYY & YYYY-YY formatting: completeness of Date improved {len(dash_ranges)/(len(df)):.0%}')
print(f'remaining: {len(df[df.cleanedDate==0])/len(df):.0%}')

YYYYYYYY & YYYYYY formatting: completeness of Date improved 6%
YYYY-YYYY & YYYY-YY formatting: completeness of Date improved 13%
remaining: 6%


In [10]:
# checking formatting of remaining 7%
df[(df.cleanedDate==0)].Date.value_counts()[2:30];

In [11]:
# cleaning date ranges yyyyyyyyyy
# keeping [0:4]
# discarding [4:6]
# stashing [6:]

multirange = df[df.Date.str.contains('\d{9}[^\D]', regex=True)].index
df.loc[multirange,'Date2'] = df.loc[multirange,'Date'].apply(lambda x: str(x)[6:])
df.loc[multirange,'cleanedDate'] = df.loc[multirange,'Date'].apply(lambda x: str(x)[0:4])
print(f'YYYYYYYYYY formatting: completeness of Date improved {len(multirange)/(len(df)):.0%}')

YYYYYYYYYY formatting: completeness of Date improved 2%


In [19]:
# checking formatting of remaining 2%
df[(df.cleanedDate==0)].Date.value_counts()[2:30]

91962            69
2015-            41
51877-221894     30
19641965-66      27
1910-111912      27
1958-641964      25
18-231966        25
19721971-1972    21
19501949-50      21
11970            21
31916            20
19271925-1927    18
71925            17
-1991            17
91907            16
61966-251967     15
17-191969        15
19611965-66      14
1923-241925      14
61965            12
13-191970        12
1978-791995      11
-1925            11
1914-211925      10
2-101969          9
10-151970         9
8-131970          9
1999-             9
Name: Date, dtype: int64

In [136]:
#processing dates formatted YYYYYYYY
nodash = df[(df.Date.str.len()==8)&(df.Date.str.contains('-')==False)].index
df.loc[nodash,'cleanedDate'] = [int(i[0:4]) for i in df.loc[nodash,'Date']]
df.loc[nodash,'Date2'] = [int(i[4:]) for i in df.loc[nodash,'Date']]

print(f'completeness of Date improved {len(nodash)/(len(df)):.0%}')

completeness of Date improved 4%


In [125]:
daterange = df[(df.Date.str.len()==7)&(df.Date.str.contains('-')==True)].index
df.loc[daterange,'cleanedDate'] = [i.split('-')[0] for i in df.loc[daterange,'Date']]
df.loc[daterange,'Date2'] = [i.split('-')[1] for i in df.loc[daterange,'Date']]
print(f'completeness of Date improved {len(daterange)/(len(df)):.0%}')
print(f'total improvements: {(len(df[df.cleanedDate!=0])-len(pfdates))/(len(df)):.0%}')
print(f'remaining: {len(df[df.cleanedDate==0])/len(df):.0%}')

completeness of Date improved 6%
total improvements: 18%
remaining: 7%


In [126]:
df.loc[dash_ranges,'cleanedDate'].unique()

array(['1976', '1980', '1989', '1915', '1982', '1', '1929', '1918',
       '1975', '1952', '1956', '1960', '1978', '1934', '1986', '1926',
       '1937', '1946', '1939', '1941', '1944', '1945', '1957', '1927',
       '1974', '1977', '1965', '1972', '1968', '1994', '1988', '1970',
       '1959', '1985', '1979', '1913', '1930', '1928', '1955', '1984',
       '1987', '1947', '1992', '1993', '1990', '1991', '1958', '1954',
       '1950', '1953', '1933', '1932', '1902', '1923', '1925', '1938',
       '1903', '1898', '1935', '1880', '1936', '1964', '1900', '1943',
       '1920', '1904', '1905', '1908', '1971', '1948', '1949', '1966',
       '1942', '1951', '1983', '1924', '1962', '1899', '1909', '1963',
       '1901', '1917', '1969', '1840', '19531950', '1931', '1961', '1967',
       '1940', '1914', '1910', '1897', '1884', '1890', '1894', '1895',
       '1973', '1877', '1919', '1896', '1911', '1916', '1891', '19381936',
       '19751968', '19121908', '19651964', '19791974', '19851980',
     

In [73]:
[int(i[0:4]) for i in df.loc[daterange,'Date']]

ValueError: invalid literal for int() with base 10: '3-41'

In [79]:
len(df[df['cleanedDate']!=0])

14147

In [50]:
[i[0:4] for i in df.loc[nodash,'Date']]

['1958',
 '1985',
 '1985',
 '1985',
 '1915',
 '1915',
 '1915',
 '1915',
 '1915',
 '1982',
 '1923',
 '1927',
 '1982',
 '1926',
 '1979',
 '1981',
 '1982',
 '1979',
 '1975',
 '1916',
 '1934',
 '1959',
 '1959',
 '1959',
 '1959',
 '1964',
 '1964',
 '1964',
 '1964',
 '1964',
 '1964',
 '1964',
 '1964',
 '1964',
 '1978',
 '1978',
 '1972',
 '1972',
 '1972',
 '1979',
 '1982',
 '1980',
 '1951',
 '1959',
 '1913',
 '1988',
 '1928',
 '1922',
 '1921',
 '1945',
 '1965',
 '1924',
 '1967',
 '1990',
 '1995',
 '1991',
 '1958',
 '1983',
 '1990',
 '1994',
 '1990',
 '1990',
 '1977',
 '1953',
 '1949',
 '1961',
 '1946',
 '1954',
 '1933',
 '1933',
 '1938',
 '1929',
 '1932',
 '1932',
 '1936',
 '1942',
 '1923',
 '1923',
 '1938',
 '1932',
 '1961',
 '1905',
 '1986',
 '1930',
 '1924',
 '1925',
 '1930',
 '1983',
 '1925',
 '1926',
 '1920',
 '1927',
 '1912',
 '1923',
 '1954',
 '1929',
 '1932',
 '1931',
 '1931',
 '1936',
 '1904',
 '1928',
 '1928',
 '1927',
 '1960',
 '1927',
 '1929',
 '1929',
 '1929',
 '1929',
 '1930',
 

In [65]:
df[(df.Date.str.len()==7)].Date.unique()[0:150]

array(['1976-77', '1980-81', '1989-91', '1915-17', '1982-86', '1929-30',
       '1918-20', '1975-79', '1982-83', '1956-57', '1960-61', '1952-53',
       '1978-84', '1960-62', '1934-36', '1986-87', '1926-36', '1934-35',
       '1937-41', '1946-47', '1939-40', '1941-43', '1944-45', '1945-46',
       '1945-51', '1957-58', '1927-29', '1975-76', '1974-77', '1978-80',
       '1965-66', '1972-73', '1968-72', '1994-98', '1988-90', '1970-75',
       '1934-37', '1959-64', '1985-89', '1979-80', '1913-14', '1930-32',
       '1928-29', '1928-30', '1955-58', '1927-35', '1988-95', '1977-78',
       '1984-89', '1987-92', '1989-94', '1984-92', '1992-93', '1992-94',
       '1982-98', '1990-92', '1990-94', '1988-94', '1991-92', '1958-87',
       '1976-79', '1978-79', '1954-58', '1950-51', '1953-55', '1933-49',
       '1939-41', '1932-33', '1944-47', '1902-04', '1934-38', '1926-29',
       '1923-28', '1925-28', '1926-27', '1930-40', '1938-39', '1903-04',
       '1935-38', '1898-99', '1987-91', '1945-55', 

In [ ]:
pfdates = df[(df.Date.str.len()==4)].index
df.loc[pfdates,'cleanedDate'] = df.loc[pfdates,'Date']

In [6]:
output = ''

[output.join(i) for i in list(set([i.replace(')','').replace('(','') for i in re.split('\)\s\(',"(American) (American) (French)") if len(i) > 0 ]))]
print(output)

In [7]:
def parse_multi_nat(input: str) -> str:
    return ' '.join(list(set([i.replace(')','').replace('(','').replace('\t','') for i in re.split('\)\s\(',input) if len(i) > 0 ]))).strip()

In [8]:
# # testing
# parse_multi_nat('(American) (American) (Brazilian) (French) () (American)')
# parse_multi_nat('(American) (American) () (American)')
parse_multi_nat('(German) (Swedish) (German)	')

'German Swedish'

In [9]:
multi_nationality = df[df.Nationality.str.contains('\) ')].index
df.loc[multi_nationality,'cleanedNationality'] = df.loc[multi_nationality,'Nationality'].apply(lambda x: parse_multi_nat(x))

In [12]:
clean_nats = df[df.cleanedNationality.isnull()].index
df.loc[clean_nats,'cleanedNationality'] = df.loc[clean_nats,'Nationality']
# df.loc[:,'cleanedNationality'] = df.loc[:,'cleanedNationality'].str.replace(' Nationality unkown ','')

In [13]:
df.cleanedNationality.unique()

array(['Austrian', 'French', '', 'American', 'German', 'Dutch', 'Italian',
       'Swedish', 'American French', 'British', 'Japanese',
       'British Dutch', 'Argentine', 'Brazilian', 'Swiss', 'Luxembourger',
       'Spanish', 'Austrian Polish', 'Russian', 'Iranian',
       'Swiss American', 'Canadian German American Dutch Belgian Spanish',
       'French German American Japanese Dutch Belgian',
       'American Japanese Dutch French', 'Dutch Norwegian',
       'Swiss French', 'Finnish', 'German American', 'German Swiss',
       'American Dutch Japanese', 'Japanese Italian', 'Canadian',
       'Nationality unknown', 'Danish', 'Belgian', 'American Italian',
       'Czech', 'Moroccan', 'Coptic', 'Persian', 'Canadian American',
       'Colombian', 'Dutch Danish', 'German Italian', 'Australian',
       'Chinese', 'Mexican', 'Slovenian', 'American Scottish',
       'German Swedish', 'Hungarian', 'British American',
       'Argentine Italian', 'American Swedish', 'Polish', 'British Swiss',


### Determining opportunities to impute Date <a class="anchor" id="date-imputation"></a>

## Extracting From Biographies <a class="anchor" id="bio-extraction"></a>
### [Extracting birthplace](#birthplace), if listed
### [Imputing nationalities](#impute-nationalities)

In [14]:
bio2ref = df[df.ArtistBio.str.contains(',')==True].index

df.loc[:,'NationalityBio'] = df.loc[:,'ArtistBio'].apply(lambda x: x.split(',')[0])
df.loc[bio2ref,'Bio2'] = df.loc[bio2ref,'ArtistBio'].apply(lambda x: x.split(',')[1])

In [15]:
df[df.Bio2.str.contains('born')==True].ArtistBio.value_counts()

American, born France. 1911–2010                                   3339
American, born Germany. 1886–1969                                  2657
American, born 1934                                                1534
French, born Belarus. 1887–1985                                    1166
American, born Lithuania. 1931–1978                                 831
                                                                   ... 
German, born Bohemia. 1875–1951                                       1
Swedish, born 1967                                                    1
Italian, born 1966 Italian, born 1972 Italian, established 1998       1
German, born 1897                                                     1
Malian, born 1953                                                     1
Name: ArtistBio, Length: 3230, dtype: int64

In [16]:
bp_ref = df[df.ArtistBio.str.contains('born')==True].index

for i in bp_ref:
    output = list(set(re.findall('born\s\d{0,}\s{0,1}([A-Za-z]+)', df.loc[i,'ArtistBio'])))
    
    if len(output) == 1:
        df.loc[i,'Birthplace'] = output[0]
    elif len(output) == 0:
        df.loc[i,'Birthplace'] = 'N/A'
    else:
        df.loc[i,'Birthplace'] = [''.join(i) for i in output][0]
        
df['Birthplace'].fillna('N/A', inplace=True)

In [17]:
[' '.join(i) for i in output][0]

'G e r m a n y'

In [18]:
df.Birthplace.value_counts()[20:40]

the          258
Canada       252
Cuba         244
British      197
Korea        170
Japan        152
India        149
Latvia       138
Bohemia      132
Estonia      129
Spain        123
Chinese      120
Sweden       120
in           117
Scotland     115
Brass        112
Swiss        112
Argentina    105
Argentine     88
Ireland       87
Name: Birthplace, dtype: int64

In [19]:
# Answer
primes = [] # Set a list to catch prime values

for i in range(3, 2000): 
    # All statement evaluates to true if all of the iterables satisfy the criteria
    # If i divided by the existing primes(x) never has a remainder of 0
    if all(i % x != 0 for x in primes):
        # Append this number to the primes list
        primes.append(i)

sum(primes)

277052

In [20]:
df[df.Birthplace=="['Uruguay', 'American', 'Argentine']"]

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Weight (kg),Width (cm),Seat Height (cm),Duration (sec.),cleanedDate,Date2,cleanedNationality,NationalityBio,Bio2,Birthplace


In [21]:
df[(df.Nationality==None)]

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Weight (kg),Width (cm),Seat Height (cm),Duration (sec.),cleanedDate,Date2,cleanedNationality,NationalityBio,Bio2,Birthplace


In [22]:
df['Nationality'].unique()

array(['Austrian', 'French', '', ..., 'Croatian    Bosnian',
       'American American Brazilian French  American',
       ' American American American American American Kenyan Mexican American American Indian American Chinese'],
      dtype=object)

In [23]:
df['Bio2'].unique()[0:100]

array([' 1841–1918', ' born 1944', ' 1876–1957', ' born Switzerland 1944',
       ' born Estonia. 1901–1974', ' 1878–1969', ' 1878–1961',
       ' 1869–1936', ' born Austria. 1933–2010', ' born 1932',
       ' born 1944 Dutch', ' 1949–1991', ' 1934–2015',
       ' 1931–1997 Italian', ' born Germany. 1886–1969', ' 1885–1940',
       ' 1885–1940 Swedish', ' 1895–1979 French', ' 1895–1979 American',
       ' born 1947', ' born Germany now Poland. 1887–1953',
       ' born Italy. 1919–2013', ' 1918–1997', ' born 1936',
       ' born Poland 1954 American', ' born Iraq. 1950–2016',
       ' born 1928', ' born 1944 British', ' 1867–1959', ' born 1935',
       ' est. 1964–1979 American', ' established 1980 American',
       ' born 1943', ' born 1941', ' born 1931', ' 1883–1931 Dutch',
       ' 1909–1994 Brazilian', ' born 1957', ' 1895–1983', ' 1909–1994',
       ' born Austria. 1900–1985', ' born 1946', ' born Poland 1946',
       ' born 1944 American', ' 1931–1997', ' born 1956', ' 1927–2016

In [24]:
df[df.Birthplace.str.contains('\[')==True].Birthplace.unique()

array([], dtype=object)

### Impute Nationalities <a class="anchor" id="impute-nationalities"></a>

In [25]:
# checking mismatches
len(df[df.Nationality!=df.NationalityBio])

15865

In [26]:
# pulling top 10 donors, by volume
[print('{:,} items from {}'.format(df.CreditLine.value_counts()[i], i)) for i in df.CreditLine.value_counts().index[0:10]];

11,258 items from The Louis E. Stern Collection
10,612 items from Gift of the artist
8,392 items from Purchase
5,438 items from The Gilbert and Lila Silverman Fluxus Collection Gift
4,929 items from Abbott-Levy Collection. Partial gift of Shirley C. Burden
2,472 items from The Judith Rothschild Foundation Contemporary Drawings Collection Gift
2,383 items from Gift of Kleiner, Bell & Co.
1,889 items from Gift of Abby Aldrich Rockefeller
1,863 items from -
1,686 items from Gift of The Judith Rothschild Foundation


In [27]:
# pulling remaining, most common formatting issues
df[df.cleanedDate==0].Date.value_counts()[0:10]

1966    2135
-       2105
1967    2098
1965    1970
1971    1925
1968    1834
1969    1687
1970    1669
1973    1662
1930    1657
Name: Date, dtype: int64

In [28]:
df[(df.Date==0)&(df.BeginDate.str.len()==4)]

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Weight (kg),Width (cm),Seat Height (cm),Duration (sec.),cleanedDate,Date2,cleanedNationality,NationalityBio,Bio2,Birthplace


In [29]:
df[(df.Date==0)|(df.Date==None)]

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Weight (kg),Width (cm),Seat Height (cm),Duration (sec.),cleanedDate,Date2,cleanedNationality,NationalityBio,Bio2,Birthplace


In [30]:
# for i in df.loc[ref, 'Date'].index:
#     f = df.loc[i,'Date'][:4]
#     s = df.loc[i,'Date'][4:]
    
#     df.loc[i,'Date'] = f
#     df.loc[i,'Date2'] = s